## Simple Client

This demos shows how to use the `bayeos.cli.SimpleClient` class to connect to a [BayEOS Server](http://www.bayceer.uni-bayreuth.de/bayeos/de/bayeos-ser/gru/html.php?id_obj=107625). The class implements basic functionalities with native python tpyes for read and write functions. There are no dependencies to external libraries.

### Import of bayeos package

The SimpleClient class can be imported as any other python class.

In [1]:
from bayeos.cli import SimpleClient

We need an instance of it:

In [2]:
bayeos = SimpleClient()

### Server Connection

Let us now open the connection to our freely available BayEOS Server and save the connection information in a encrypted file on our local disk. We can pick the connection information later on by an alias.

In [3]:
bayeos.connect(url="http://bayeos.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet",user="gast",password="gast",save_as="guest@bayeos")

Open connection to http://bayeos.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet as user gast


True

A connection can be closed by calling `disconnect()`

In [4]:
bayeos.disconnect()

Connection closed


True

Just call the `connect()` function with the parameter `listConnections=True` to list all available connection alias.

In [5]:
bayeos.connect(listConnections=True)

Alias               |URL                                                                             |User      
--------------------|--------------------------------------------------------------------------------|----------
root@bayeos         |https://bayeos.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet                 |root      
root@localhost      |http://localhost:5532/XMLServlet                                                |root      
bayeos              |https://bayeos.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet                 |bt220259  
me@bayeos           |https://bayeos.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet                 |niessner  
guest@bayeos        |http://bayeos.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet                  |gast      
myAlias             |http://bayconf.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet                 |root      


True

Use the alias to reopen the connection. The following call retrieves the alias in `$HOME/.bayeos.pwd` and decryptes it with a key stored in `$HOME/.bayeos.pwd_key`.

In [6]:
bayeos.connect(url="guest@bayeos")

Open connection to http://bayeos.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet as user gast


True

### Basic Navigation

The basic navigation interface is loosely based on the `UNIX` file sytem navigation commands.

We can list the content of a working directory by calling the `ls()` function. The output shows the id, name, type, first and last result date of folders and series rendered as a table.

In [7]:
bayeos.ls()

Alle Messungen
None      |..
126232    |Dr. Hans-Frisch                                   |messung_ordner      |1884-01-26 17:20:00 |2020-03-02 14:50:00 
14333     |Micrometeorology Dept                             |messung_ordner      |1984-11-01 18:25:00 |2020-03-02 15:02:00 
117846    |Pivot Tabellen                                    |messung_ordner      |1970-01-01 00:00:00 |2020-01-01 00:00:00 
14325     |Steinkreuz                                        |messung_ordner      |1994-03-17 12:00:00 |2008-04-08 10:50:00 


Use the `cd()` function to change the current working directory. You need to pass in the folder id.

In [8]:
bayeos.cd(14333)

Alle Messungen/Micrometeorology Dept
83        |..
14473     |AWS Ecological Botanical Garden                   |messung_ordner      |1992-01-06 00:00:00 |2020-03-02 14:56:00 


Please call `cd('..')` to switch into the parent directory.

In [ ]:
bayeos.cd('..')

The `pwd()` function prints the current working directory path.

In [ ]:
bayeos.pwd()

Please close the connection before you shut down the kernel.

In [9]:
bayeos.disconnect()

Connection closed


True

## CRUD Operations

The SimpleClient provides methods to create, read, update and delete a series.

Import the package and open a new connection with write privileges.

In [11]:
from bayeos.cli import SimpleClient 
bayeos = SimpleClient()
bayeos.connect(url='myAlias')

Open connection to http://bayconf.bayceer.uni-bayreuth.de/BayEOS-Server/XMLServlet as user root


True

### Create series

To create three series and return the ids as a list we use pythons handy list comprehension functionality.

In [12]:
ids = [bayeos.createSeries("Dummy" + x) for x in ["A","B","C"] ]

Import some observation records as list of lists. The first column contains the observation time by convention.

In [13]:
from datetime import datetime, timedelta
now = datetime.utcnow()
past = now + timedelta(minutes=-10)
data = [[now,2.1,2.2,2.3],[past,1.1,1.2,1.3]]
bayeos.writeSeries(ids,data)

6 records imported.


### Read values

In [14]:
(header, data) = bayeos.getSeries(ids,interval='today')

In [15]:
header

['DummyA', 'DummyB', 'DummyC']

In [16]:
data

[[datetime.datetime(2020, 3, 2, 13, 56, 15, tzinfo=<UTC>),
  1.100000023841858,
  1.2000000476837158,
  1.2999999523162842],
 [datetime.datetime(2020, 3, 2, 14, 6, 15, tzinfo=<UTC>),
  2.0999999046325684,
  2.200000047683716,
  2.299999952316284]]

### Update values

You can call the writeSeries function with `overwrite=True` to update a value.

In [17]:
data = [[now,3.1,3.2,3.3]]
bayeos.writeSeries(ids,data,overwrite=True)
(header, data) = bayeos.getSeries(ids,interval='today')

3 records imported.


In [18]:
data

[[datetime.datetime(2020, 3, 2, 13, 56, 15, tzinfo=<UTC>),
  1.100000023841858,
  1.2000000476837158,
  1.2999999523162842],
 [datetime.datetime(2020, 3, 2, 14, 6, 15, tzinfo=<UTC>),
  3.0999999046325684,
  3.200000047683716,
  3.299999952316284]]

### Delete series

Drop all created series by id.

In [19]:
[bayeos.deleteSeries(x) for x in ids] 

[True, True, True]

### Close the connection

In [ ]:
bayeos.disconnect()